# 3-1. Identify Job Skills from Job Posting

Job posting data is provided by: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings

Author: Yu Kyung Koh

Last Updated: 2025/05/16

In this code, I am going to identify job skills from job posting data. Here, I utilize the list of job skills compiled by Lightcast (downloaded from https://lightcast.io/open-skills)


## SECTION 1: Bring in the list of job skills compiled by Lightcast + clean them

In [3]:
import pandas as pd
import os
import re
from tqdm import tqdm
import multiprocessing 
from joblib import Parallel, delayed

import nltk
from nltk.corpus import stopwords
#from rapidfuzz import process, fuzz

In [4]:
### Bring in lightcast skills data 
lightcastdir = '/Users/yukyungkoh/Desktop/1_Post-PhD/7_Python-projects/2_practice-NLP_job-posting_NEW/2_data/Lightcast'
skills_file = os.path.join(lightcastdir, 'lightcast_skills.csv')
skills_df = pd.read_csv(skills_file)

In [5]:
skills_df.head()

,skill_name,skill_category
0,.NET Assemblies,Specialized Skill
1,.NET Development,Specialized Skill
2,.NET Framework,Specialized Skill
3,.NET Framework 1,Specialized Skill
4,.NET Framework 3,Specialized Skill


In [6]:
skills_df["skill_category"].value_counts()  

skill_category
Specialized Skill    30488
Certification         3453
Common Skill           471
Name: count, dtype: int64

In [7]:
### Extract content inside parentheses into a new column
skills_df['skill_parentheses'] = skills_df['skill_name'].str.extract(r'\((.*?)\)')

In [8]:
### Remove everything in parentheses from the 'skill' column (e.g. "Python (Programming Language)")
skills_df['skill_name'] = skills_df['skill_name'].str.replace(r'\s*\(.*?\)', '', regex=True)

In [9]:
### Examine if skill_parentheses contain generic terms 
skills_df[skills_df['skill_parentheses'] == 'Business']

,skill_name,skill_category,skill_parentheses
26850,Restructuring,Specialized Skill,Business
31693,Transaction Processing,Specialized Skill,Business
32015,Turnkey,Specialized Skill,Business


In [10]:
skills_df[skills_df['skill_name'] == 'Business']

,skill_name,skill_category,skill_parentheses


In [11]:
# Combine 'skill_name' and 'skill_parentheses' to a single list of skill keywords 
#    combined_skills = pd.concat([
#        skills_df['skill_name'],
#        skills_df['skill_parentheses']
#    ])
## => Decided not to do this, because skills_parentheses contain a lot of generic keywords like "business" that does not help us identify skills

In [12]:
### --- STEP 5: Drop duplicates and missing values 
skills_df = skills_df.dropna(subset=['skill_name'])
skills_df = skills_df.drop_duplicates(subset='skill_name')

In [13]:
# Convert to lowercase and strip whitespace
skills_df['skill_name'] = skills_df['skill_name'].str.lower().str.strip()

In [14]:
skills_df["skill_category"].value_counts()  

skill_category
Specialized Skill    30467
Certification         3448
Common Skill           471
Name: count, dtype: int64

In [15]:
## Create a list of "Specialized Skill and Certification" and "Common Skill" 
specialized_skills = skills_df[
    skills_df['skill_category'].isin(['Specialized Skill', 'Certification'])
]['skill_name'].dropna().unique().tolist()

common_skills = skills_df[
    skills_df['skill_category'] == 'Common Skill'
]['skill_name'].dropna().unique().tolist()

In [16]:
## Augment the list to include (AI, ML, NLP, SQL, API, etc.) 
specialized_skills = list(set(specialized_skills + ['ai', 'ml', 'nlp', 'sql', 'api', 'aws', 'microsoft suite', 'seo tools']))  # this ensures that there are no duplicates 

In [17]:
## Remove terms like "make" from the skills_list, because it confuses the skills with a commonly used verb
specialized_skills = [s for s in specialized_skills if s.lower() not in {'make', 'track', 'disabilities', 'diversity and inclusion', 'transparency'}]
#specialized_skills

In [18]:
## ----- Allowing for common spacing mistakes in the job posting ----- ##
# Manual spacing error aliases
alias_map = {
    "photo shop": "photoshop",
    "power point": "powerpoint",
    "word press": "wordpress",
    "excel sheet": "excel",
    # Add more as you find
}

# Add alias keys to skills list
specialized_skills += list(alias_map.keys())


# SECTION 2: Bring in the job posting data 

In [20]:
## Import cleaned data from STEP 2-1
cleandatadir = '/Users/yukyungkoh/Desktop/1_Post-PhD/7_Python-projects/2_practice-NLP_job-posting_NEW/2_data/cleaned_data'
jobdata = os.path.join(cleandatadir, '2_job-posting_cleaned_df.pkl')
jobs_df = pd.read_pickle(jobdata, 'zip')


In [21]:
## Check how many job postings are in this data 
len(jobs_df)

29724

In [22]:
## Lower case
jobs_df['combined_desc'] = jobs_df['combined_desc'].apply(lambda x: " ".join(x.lower()for x in x.split()))

In [23]:
jobs_df.head(10)

,job_id,company_name,title,work_type,normalized_salary,combined_desc,job_category
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,FULL_TIME,38480.0,leading real estate firm new jersey seeking ad...,Marketing
2,10998357,The National Exemplar,Assitant Restaurant Manager,FULL_TIME,55000.0,national exemplar accepting application assist...,Other Manager
12,56482768,NaN,Appalachian Highlands Women's Business Center,FULL_TIME,NaN,full – program director appalachian highland w...,Business/Finance Job
14,69333422,Staffing Theory,Senior Product Marketing Manager,FULL_TIME,NaN,leading pharmaceutical company committed devel...,Marketing
18,111513530,United Methodists of Greater New Jersey,"Content Writer, Communications",FULL_TIME,NaN,"application opening date: april 24, 2024 title...",Marketing
22,133130219,NaN,Software Engineer,FULL_TIME,NaN,"education bachelor's degree software, math, sc...",Software/Developer
26,175485704,GOYT,Software Engineer,PART_TIME,NaN,seeking skilled motivated remote software deve...,Software/Developer
35,266566927,Revesco Properties,Marketing & Office Coordinator,FULL_TIME,65000.0,revesco properties:revesco property boutique c...,Marketing
46,805229245,NaN,"Manager, Retail Pharmacy",FULL_TIME,137500.0,operation supervises departmental distribution...,Other Manager
47,844454682,NaN,Commercial Property Manager,FULL_TIME,NaN,gordon partner (www.gordonpartners.com) seekin...,Other Manager


In [24]:
# Checking an entry in "combined_desc_corrected"
jobs_df["combined_desc"][22]
    ## Typo: photo shop -> photoshop

"education bachelor's degree software, math, science required skill analytical skills, group work, knowledge intended audience, understanding different role"

# SECTION 3: Job skill extraction using skills_list

In [26]:
# -----------------------------
# Step 1: Define your skill extractor
# -----------------------------
# Clean and escape all skills (Only considering specialized_skills for now, not common_skills)
skills_cleaned = [re.escape(skill.strip().lower()) for skill in specialized_skills if isinstance(skill, str)]

# Combine into a single regex: match whole word or phrase using \b boundaries
skills_pattern = r'\b(?:' + '|'.join(skills_cleaned) + r')\b'
skills_regex = re.compile(skills_pattern, flags=re.IGNORECASE)

def extract_skills(text):
    if not isinstance(text, str):
        return []
    matches = skills_regex.findall(text.lower())
    return list(set(matches))  # remove duplicates

In [27]:
# -----------------------------
# Step 2: Try running it for the jobs_df 
# -----------------------------
# Apply this for the first entry only. 
extract_skills(jobs_df.loc[0, "combined_desc"])
print(extract_skills(jobs_df.loc[0, "combined_desc"]))

['brand strategy', 'genetics', 'graphic design', 'marketing', 'branding', 'event planning', 'brokerage', 'online advertising', 'jersey', 'social media', 'adobe creative cloud', 'real estate development', 'email marketing']


In [28]:
## Apply this to the whole data
tqdm.pandas()
#jobs_df['matched_skills'] = jobs_df['combined_desc_corrected'].progress_apply(extract_skills)
    # => Takes too much time (over 4 hours) to run 

In [29]:
# -----------------------------
# Step 3: Batch + Parallel Processing for 1 Batch Only
# -----------------------------
batch_size = 200
n_jobs = multiprocessing.cpu_count() - 1

# First batch only (0:200)
batch = jobs_df.iloc[0:batch_size]

# Parallel skills extraction
batch_skills = Parallel(n_jobs=n_jobs)(
    delayed(extract_skills)(text) for text in batch["combined_desc"]
)
    
# Create a Series with the correct original indices
batch_skills_series = pd.Series(batch_skills, index=batch.index)

# Save it into a temporary column
jobs_df.loc[batch.index, "skills_temp"] = batch_skills_series

## Check if the first 20 rows have been coded correctly 
jobs_df[["combined_desc", "skills_temp"]].head(10)

,combined_desc,skills_temp
0,leading real estate firm new jersey seeking ad...,"[brokerage, brand strategy, marketing, social ..."
2,national exemplar accepting application assist...,[restaurant management]
12,full – program director appalachian highland w...,"[constant contact, indicators, business market..."
14,leading pharmaceutical company committed devel...,"[market environment, market research, gastroen..."
18,"application opening date: april 24, 2024 title...","[photography, wordpress, seo tools, inquiry, m..."
22,"education bachelor's degree software, math, sc...",[]
26,seeking skilled motivated remote software deve...,"[object-oriented programming, mysql, javascrip..."
35,revesco properties:revesco property boutique c...,"[business writing, marketing strategies, event..."
46,operation supervises departmental distribution...,[pharmacy experience]
47,gordon partner (www.gordonpartners.com) seekin...,"[vendor contracts, reconciliation, property ma..."


In [30]:
jobs_df.iloc[5]["combined_desc"]

"education bachelor's degree software, math, science required skill analytical skills, group work, knowledge intended audience, understanding different role"

In [31]:
jobs_df.iloc[5]["skills_temp"]

[]

In [32]:
jobs_df.drop(columns=['skills_temp'], inplace=True)

In [33]:
# -----------------------------
# Step 4: Batch + Parallel Processing for ALL Job Postings
# -----------------------------
batch_size = 200
matched_skills_all = []
n_jobs = multiprocessing.cpu_count() - 1

for i in tqdm(range(0, len(jobs_df), batch_size), desc="Extracting Skills"):
    batch = jobs_df.iloc[i:i+batch_size]
    
    batch_skills = Parallel(n_jobs=n_jobs)(
        delayed(extract_skills)(text) for text in batch["combined_desc"]
    )
    
    matched_skills_all.append(pd.Series(batch_skills, index=batch.index))

# Recombine without losing index
matched_full = pd.concat(matched_skills_all)

# Assign corrected results correctly back
jobs_df["skills_extracted"] = matched_full

## => Takes about 1.5 hours to run. 

Extracting Skills: 100%|██████████████████████| 149/149 [53:11<00:00, 21.42s/it]


In [34]:
## Save the data 
cleandatadir = '/Users/yukyungkoh/Desktop/1_Post-PhD/7_Python-projects/2_practice-NLP_job-posting_NEW/2_data/cleaned_data'
savefile = os.path.join(cleandatadir, '3_job-posting_skills-extracted_df.pkl')
jobs_df.to_pickle(savefile, compression='zip')